In [ ]:
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score

# Define paths and parameters
dir_path = 'data/images/'
train_path = 'data/images/train/'
classes = ['0', '1']
image_size = (224, 224)
batch_size = 32
num_epochs = 10  # Increase number of epochs

# Data augmentation and normalization
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    validation_split=0.2
)

# Load and prepare the train and validation datasets
train_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=classes,
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    train_path,
    target_size=image_size,
    batch_size=batch_size,
    class_mode='categorical',
    classes=classes,
    subset='validation'
)

# Define the model architecture
base_model = tf.keras.applications.ResNet50(
    include_top=False,
    weights='imagenet',
    input_shape=(224, 224, 3)
)

for layer in base_model.layers[:-10]:  # Unfreeze last 10 layers
    layer.trainable = True

x = layers.GlobalAveragePooling2D()(base_model.output)
x = layers.Dense(256, activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(len(classes), activation='softmax')(x)

model = tf.keras.models.Model(base_model.input, output)

# Compile the model with a lower learning rate
model.compile(optimizer=tf.keras.optimizers.Adam(lr=1e-4),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Train the model with more epochs
model.fit(
    train_generator,
    epochs=num_epochs,
    validation_data=validation_generator
)

# Evaluate the model
validation_steps = validation_generator.samples // batch_size + 1
predictions = model.predict(validation_generator, steps=validation_steps)
predicted_classes = tf.argmax(predictions, axis=1)
true_classes = validation_generator.classes
accuracy = accuracy_score(true_classes, predicted_classes)

print("Test Accuracy:", accuracy)


In [ ]:
# Save the trained model to disk
model.save('trained_model.h5')